In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geoenrichment import *
import pandas as pd
from pathlib import Path

In [2]:
data_p = Path("output")

In [3]:

#gis = GIS(username='izaslavsky_ucsd')  # this will ask for password. 
print("Enter the Client ID/APP ID generated from Arc GIS:")
app_id = input()
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)
arcgis.__version__
# Elm5V3upnnV17Q3r

Enter the Client ID/APP ID generated from Arc GIS:


 Elm5V3upnnV17Q3r


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=eeZyhKZCqEFTJvbDWhe5Nfg1Np70IW&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'2.1.0'

### What information is available for your country of interest?

In [4]:
# What is available for the US?
usa = Country.get('US')

In [5]:
# Read excel file with sheet name
dict_df = pd.read_excel('Variables to Use in Food Swamp and Opportunities Maps 2-2-23.xlsx', 
                   sheet_name=['Esri Consumer Spending Data ','Esri Business Data','Esri Market Potential Data','Esri Demographics'])

In [6]:
# consumer_spending_vars_df = dict_df.get('Esri Consumer Spending Data ')
# list(consumer_spending_vars_df['Variables to use'].unique())

In [7]:
segment_list = list(dict_df.keys())

In [15]:
segment_list

['Esri Consumer Spending Data ',
 'Esri Business Data',
 'Esri Market Potential Data',
 'Esri Demographics']

In [8]:
subgeographies_to_enrich = ['block_groups','tracts','zip5']
counties_scoped = ['San_Diego_County','Imperial_County']

In [9]:
def create_dir(directory):
    directory.mkdir(exist_ok=True)

In [10]:
#TEst Create dir
# enrichment_path = data_p / "San_Diego_County"
# create_dir(enrichment_path)

In [11]:
def parse_enrichment_segment(segment):
    print(f"{segment}:")
    vars_df = dict_df.get(segment)
    vars_df = vars_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    # drop Nan from the dataframe
    vars_df = vars_df.dropna(subset=['Variables to use'])
    var_list = list(vars_df['Variables to use'].unique())
    print(f"\tNumber of Variables: {len(var_list)}")
    for cnty in counties_scoped:
        create_dir(data_p / cnty)
        for sg in subgeographies_to_enrich:
            create_dir(data_p / cnty / sg)
            enrich_geography(segment,cnty,sg,var_list)

In [13]:
def enrich_geography(segment,county,subgeography,variable_list):
    segment_name = segment.lower().replace(" ", "_")
    enrichment_file_name = f"{segment_name}.csv"
    enrichment_path = f"output/{county}/{subgeography}/{enrichment_file_name}"

    if subgeography=='block_groups':
        subgeographies = usa.subgeographies.states['California'].counties[county].block_groups
    elif subgeography=='tracts':
        subgeographies = usa.subgeographies.states['California'].counties[county].tracts
    elif subgeography=='zip5':
        subgeographies = usa.subgeographies.states['California'].counties[county].zip5
    
    enriched_df = enrich(study_areas=subgeographies, analysis_variables=variable_list)
    enriched_df.to_csv(enrichment_path,index=False)
    
    print("\tEnrichment Details:")
    print(f"\t\tNumber of rows/{subgeography}: {enriched_df.shape[0]}")
    print(f"\t\tNumber of columns/variables: {enriched_df.shape[1]}")
          
    print(f"\tFile Saved: {enrichment_file_name}")
    print(f"\tFile Saved Path: {enrichment_path}")
    
    print("\n")

In [14]:
%time
for s in segment_list:
    parse_enrichment_segment(s)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs
Esri Consumer Spending Data :
	Number of Variables: 360
	Enrichment Details:
		Number of rows/block_groups: 1000
		Number of columns/variables: 369
	File Saved: esri_consumer_spending_data_.csv
	File Saved Path: output/San_Diego_County/block_groups/esri_consumer_spending_data_.csv


	Enrichment Details:
		Number of rows/tracts: 736
		Number of columns/variables: 369
	File Saved: esri_consumer_spending_data_.csv
	File Saved Path: output/San_Diego_County/tracts/esri_consumer_spending_data_.csv


	Enrichment Details:
		Number of rows/zip5: 114
		Number of columns/variables: 369
	File Saved: esri_consumer_spending_data_.csv
	File Saved Path: output/San_Diego_County/zip5/esri_consumer_spending_data_.csv


	Enrichment Details:
		Number of rows/block_groups: 114
		Number of columns/variables: 369
	File Saved: esri_consumer_spending_data_.csv
	File Saved Path: output/Imperial_County/block_groups/esri_consumer_spending_data_.cs